## Part 5. 상태 커스터마이징 실습

목표
- 웹 검색 도구를 통합한 챗봇을 구축하고, 상태 replay 기능을 활용하여 검색어를 수정하고 재실행하는 방법을 학습합니다.
- 첫 번째 검색: `기계식 키보드 추천`으로 일반 검색을 수행합니다.
- 두 번째 검색: replay 기능을 사용하여 검색어를 `기계식 키보드 추천 2025 site:quasarzone.com`으로 수정하고 재실행합니다.

요구사항
- `TavilySearch` 도구를 설정하고 LLM에 바인딩하세요.
- 메모리 기능이 있는 그래프를 구축하세요.
- 상태 히스토리를 확인하고 `update_tool_call`을 사용하여 검색어를 수정하세요.
- 수정된 상태로 그래프를 재실행하세요.

In [ ]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)

# 프로젝트 이름
logging.langsmith("LangGraph-Exercises")

In [ ]:
# 준비 코드
from typing import Annotated
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver
from langchain_teddynote.messages import stream_graph
from langchain_teddynote.graphs import visualize_graph

### 1. 웹 검색 도구 설정

- `TavilySearch` 도구를 설정하세요.
- 최대 검색 결과 수는 2개로 설정하세요.

In [ ]:
# 실습 코드
from langchain_tavily import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition

# TODO: TavilySearch 도구를 설정하세요 (max_results=2)
tool = # 코드 입력
tools = # 코드 입력

In [ ]:
# 정답 코드
from langchain_tavily import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition

# TODO: TavilySearch 도구를 설정하세요 (max_results=2)
tool = TavilySearch(max_results=2)
tools = [tool]

### 2. LLM과 State 정의

- ChatOpenAI 모델을 생성하고 도구를 바인딩하세요.
- 모델명: `gpt-4.1`, 온도: `0`
- 그래프 State를 정의하세요.

In [ ]:
# 실습 코드
# TODO: LLM 모델을 생성하고 도구를 바인딩하세요
llm = # 코드 입력
llm_with_tools = # 코드 입력

# TODO: State를 정의하세요
class State(TypedDict):
    messages: # 코드 입력

In [ ]:
# 정답 코드
# TODO: LLM 모델을 생성하고 도구를 바인딩하세요
llm = ChatOpenAI(model="gpt-4.1", temperature=0)
llm_with_tools = llm.bind_tools(tools)


# TODO: State를 정의하세요
class State(TypedDict):
    messages: Annotated[list, add_messages]

### 3. 챗봇 노드와 그래프 구성

- 챗봇 노드를 작성하세요.
- ToolNode를 생성하세요.
- StateGraph를 구성하고 노드와 엣지를 추가하세요.
- 메모리 기능을 추가하여 컴파일하세요.

In [ ]:
# 실습 코드
# TODO: 챗봇 노드를 작성하세요
def chatbot(state: State):
    # 코드 입력
    pass

# TODO: 그래프를 구성하세요
builder = # 코드 입력
builder.add_node(# 코드 입력)

# TODO: ToolNode를 추가하세요
tool_node = # 코드 입력
builder.add_node(# 코드 입력)

# TODO: 엣지를 추가하세요
builder.add_conditional_edges(# 코드 입력)
builder.add_edge(# 코드 입력)
builder.add_edge(# 코드 입력)

# TODO: 메모리와 함께 컴파일하세요
memory = # 코드 입력
graph = # 코드 입력

In [ ]:
# 정답 코드
# TODO: 챗봇 노드를 작성하세요
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    return {"messages": [message]}


# TODO: 그래프를 구성하세요
builder = StateGraph(State)
builder.add_node("chatbot", chatbot)

# TODO: ToolNode를 추가하세요
tool_node = ToolNode(tools=tools)
builder.add_node("tools", tool_node)

# TODO: 엣지를 추가하세요
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")
builder.add_edge(START, "chatbot")

# TODO: 메모리와 함께 컴파일하세요
memory = InMemorySaver()
graph = builder.compile(checkpointer=memory)

### 4. 첫 번째 검색 실행

- `기계식 키보드 추천`이라는 검색어로 첫 번째 실행을 수행하세요.
- thread_id는 `keyboard-001`로 설정하세요.

In [ ]:
# 실습 코드
# TODO: 첫 번째 검색을 실행하세요
config = # 코드 입력
inputs = # 코드 입력

# 그래프 실행
stream_graph(graph, inputs=inputs, config=config)

In [ ]:
# 정답 코드
# TODO: 첫 번째 검색을 실행하세요
config = {"configurable": {"thread_id": "keyboard-001"}}
inputs = {"messages": [HumanMessage(content="기계식 키보드 추천 정보를 찾아주세요.")]}

# 그래프 실행
stream_graph(graph, inputs=inputs, config=config)

### 5. 상태 히스토리 확인 및 Replay 대상 찾기

- 상태 히스토리를 확인하고 도구 호출이 있는 체크포인트를 찾으세요.
- 해당 체크포인트를 `to_replay` 변수에 저장하세요.

In [ ]:
# 실습 코드
# TODO: 상태 히스토리를 확인하고 replay할 체크포인트를 찾으세요
to_replay = None

for state in graph.get_state_history(config):
    # 도구 호출이 있는 상태를 찾으세요
    if state.next == ("tools",) and to_replay is None:
        # 코드 입력
        break

if to_replay:
    print(
        f"✅ Replay할 체크포인트 ID: {to_replay.config['configurable']['checkpoint_id']}"
    )

In [ ]:
# 정답 코드
# TODO: 상태 히스토리를 확인하고 replay할 체크포인트를 찾으세요
to_replay = None

for state in graph.get_state_history(config):
    # 도구 호출이 있는 상태를 찾으세요
    if state.next == ("tools",) and to_replay is None:
        to_replay = state
        break

if to_replay:
    print(
        f"✅ Replay할 체크포인트 ID: {to_replay.config['configurable']['checkpoint_id']}"
    )

### 6. 검색어 수정 및 재실행

- `update_tool_call` 함수를 사용하여 검색어를 수정하세요.
- 새로운 검색어: `기계식 키보드 추천 2025 site:quasarzone.com`
- 수정된 상태로 그래프를 재실행하세요.

In [ ]:
# 실습 코드
from langchain_teddynote.tools import update_tool_call

# TODO: 검색어를 수정하세요
updated_message = update_tool_call(
    # 코드 입력 - 마지막 메시지
    tool_name="tavily_search",
    tool_args=# 코드 입력 - 새로운 검색어
)

# TODO: 수정된 메시지로 상태를 업데이트하고 재실행하세요
graph.update_state(
    to_replay.config,
    {"messages": # 코드 입력}
)

# 수정된 상태에서 재실행
stream_graph(graph, inputs=None, config=to_replay.config)

In [ ]:
# 정답 코드
from langchain_teddynote.tools import update_tool_call

# TODO: 검색어를 수정하세요
updated_message = update_tool_call(
    to_replay.values["messages"][-1],
    tool_name="tavily_search",
    tool_args={
        "query": "기계식 키보드 추천 2025 site:quasarzone.com",
        "search_depth": "basic",
    },
)

# TODO: 수정된 메시지로 상태를 업데이트하고 재실행하세요
updated_state = graph.update_state(to_replay.config, {"messages": [updated_message]})

# 수정된 상태에서 재실행
stream_graph(graph, inputs=None, config=updated_state)